# ChIP-seq peak calling
each bam file was prepared by mapping fastq files to bowtie2 index
```bash
$index = path/to/reference/genome.fasta
$chip_data = path/to/chip-seq/reads.fastq
bowtie2-build $index
bowtie2 -p 40 -q --very-sensitive-local \
-x $index \
-U $chromatin \
-S unsorted.sam

```
and then sorted and filtered
```bash
for f in *.unsorted.sam; do samtools view -@40 -m 1G -h -b -S -o ${f/.sam}.bam $f; done
for f in *.bam; do sambamba sort -t 20 -o ${f/_unsorted.bam}.sorted.bam $f; done
for f in *.sorted.bam; do sambamba index -t 20 $f; done
```


In [2]:
import os
import pybedtools


In [2]:
bam_dir = "data/chip-seq/bam_files/"
epic_output_dir = "data/chip-seq/epic_peaks/"
macs_output_dir = "data/chip-seq/macs_peaks/"
consensus_output_dir = "data/chip-seq/consensus_peaks/"

def macs_peak_calling(enriched, input, output_dir=None, target_name = None, genome_size = 800_000_000):
    """
    Call peaks using MACS3
    """
    if output_dir is None:
        output_dir = enriched.replace(".bam", "_macs_peaks")
    
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    log = os.path.join(output_dir, "macs.log")
    if target_name is None:
        target_name = os.path.basename(enriched).split("_")[0]
    
    command = f"macs3 callpeak -t {enriched} -c {input} -f BAM -g {genome_size} --broad --min-length 1000 --broad-cutoff 0.1 -n {target_name} --outdir {output_dir} > {log} 2>&1 || true"
    print(command)
    os.system(command)

def epic_peak_calling(enriched, input, output_prefix=None, chr_sizes='data/chr_sizes/Lsyl_chr_sizes.tsv'):
    """
    Call peaks using EPIC2
    NOTE: epic2 windows end coordinates can be out of chromosome size range
    """
    if output_prefix:
        output_path = output_prefix + ".tsv"
    else:
        output_path = enriched.replace(".bam", "_epic_peaks.tsv")
    
    log = output_path.replace(".tsv", ".log")
    # check if output_path parent directory exists
    output_dir = os.path.dirname(output_path)
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    command = f"epic2 -t {enriched} -c {input} --chromsizes {chr_sizes} -m 0 --output {output_path} >{log} 2>&1 || true"
    print(command)
    os.system(command)
    # convert to bed
    command = f"awk '{{print $1\"\t\"$2\"\t\"$3}}' {output_path} > {output_path.replace('.tsv', '.bed')}"
    print(command)
    os.system(command)

def merge_peaks(input_bed, output_bed=None, distance=50_000):
    """
    Merge peaks using pybedtools
    """
    if output_bed is None:
        output_bed = input_bed.replace(".bed", "_merged" + str(distance) + ".bed")
    # check if output_bed parent directory exists
    output_dir = os.path.dirname(output_bed)
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    # merge peaks
    peaks = pybedtools.BedTool(input_bed)
    peaks_merged = peaks.merge(d=distance)
    peaks_merged = peaks_merged.sort()
    peaks_merged.saveas(output_bed)

def find_consensus(macs_bed, epic_bed, output_bed=None):
    """
    Find consensus peaks between MACS3 and EPIC2
    """
    if output_bed is None:
        output_bed = macs_bed.replace(".bed", "_peaks.bed")
    # check if output_bed parent directory exists
    output_dir = os.path.dirname(output_bed)
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    # find consensus peaks
    macs_peaks = pybedtools.BedTool(macs_bed)
    epic_peaks = pybedtools.BedTool(epic_bed)
    consensus_peaks_macs = macs_peaks.intersect(epic_peaks, u=True)
    consensus_peaks_epic = epic_peaks.intersect(macs_peaks, u=True)
    consensus_peaks = consensus_peaks_macs.cat(consensus_peaks_epic, postmerge=True)
    consensus_peaks = consensus_peaks.sort()
    consensus_peaks.saveas(output_bed)

def call_peaks(enriched, input, replicate, merging_distance=50_000):
    """
    Call peaks using MACS3 and EPIC2
    """
    name = enriched + '_' + input + '_' + replicate
    macs_peak_calling(os.path.join(bam_dir, enriched + replicate + ".bam"), os.path.join(bam_dir, input + replicate +".bam"), output_dir=os.path.join(macs_output_dir, name), target_name=name, genome_size=800_000_000)
    epic_peak_calling(os.path.join(bam_dir, enriched + replicate + ".bam"), os.path.join(bam_dir, input + replicate +".bam"), output_prefix=os.path.join(epic_output_dir, name))
    merge_peaks(macs_output_dir + name + '/' + name + '_peaks.broadPeak', macs_output_dir + name + '/' + name + '_macs_merged.bed', merging_distance)
    merge_peaks(epic_output_dir + name + '.bed', epic_output_dir + name + '_epic_merged.bed', merging_distance)
    find_consensus(macs_output_dir + name + '/' + name + '_macs_merged.bed', epic_output_dir + name + '_epic_merged.bed', consensus_output_dir + name + '_peaks.bed')

def intersect_replicates(enriched, input, output_bed=None):
    """
    Intersect peaks between replicates
    """
    name = enriched + '_' + input
    if output_bed is None:
        output_bed = consensus_output_dir + name + '_intersected_replicates.bed'
    
    # check if output_bed parent directory exists
    output_dir = os.path.dirname(output_bed)
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    # intersect replicates
    R1 = pybedtools.BedTool(consensus_output_dir + name + '_R1_peaks.bed')    
    R2 = pybedtools.BedTool(consensus_output_dir + name + '_R2_peaks.bed')
    intersected = R1.intersect(R2).sort().saveas(output_bed)
    not_common_R1 = R1.intersect(R2, v=True).sort()
    not_common_R2 = R2.intersect(R1, v=True).sort()
    count_of_all_peaks = len(R1) + len(R2)
    count_of_not_common_peaks = len(not_common_R1) + len(not_common_R2)
    print(f"Proportion of deleted peaks in {name}: {count_of_not_common_peaks / count_of_all_peaks}")
    


# CenH3
rabbit antibody - control noAbA

In [39]:
enriched = 'CenH3'
input = ['input', 'noaba']
replicate = ['R1', 'R2']
merging_distance = 50_000

for i in input:
    for r in replicate:
        call_peaks(enriched, i, r, merging_distance)
    intersect_replicates(enriched, i)




macs3 callpeak -t data/chip-seq/bam_files/CenH3R1.bam -c data/chip-seq/bam_files/inputR1.bam -f BAM -g 800000000 --broad --min-length 1000 --broad-cutoff 0.1 -n CenH3_input_R1 --outdir data/chip-seq/macs_peaks/CenH3_input_R1 > data/chip-seq/macs_peaks/CenH3_input_R1/macs.log 2>&1 || true
epic2 -t data/chip-seq/bam_files/CenH3R1.bam -c data/chip-seq/bam_files/inputR1.bam --chromsizes data/chr_sizes/Lsyl_chr_sizes.tsv -m 0 --output data/chip-seq/epic_peaks/CenH3_input_R1.tsv >data/chip-seq/epic_peaks/CenH3_input_R1.log 2>&1 || true
awk '{print $1"	"$2"	"$3}' data/chip-seq/epic_peaks/CenH3_input_R1.tsv > data/chip-seq/epic_peaks/CenH3_input_R1.bed
macs3 callpeak -t data/chip-seq/bam_files/CenH3R2.bam -c data/chip-seq/bam_files/inputR2.bam -f BAM -g 800000000 --broad --min-length 1000 --broad-cutoff 0.1 -n CenH3_input_R2 --outdir data/chip-seq/macs_peaks/CenH3_input_R2 > data/chip-seq/macs_peaks/CenH3_input_R2/macs.log 2>&1 || true
epic2 -t data/chip-seq/bam_files/CenH3R2.bam -c data/chip-

# H3K4me3

rabbit antibody - control noAbA


In [3]:
enriched = 'H3K4'
input = ['input', 'noaba']
replicate = ['R1', 'R2']
merging_distance = 50_000

for i in input:
    for r in replicate:
        call_peaks(enriched, i, r, merging_distance)
    intersect_replicates(enriched, i)

macs3 callpeak -t data/chip-seq/bam_files/H3K4R1.bam -c data/chip-seq/bam_files/inputR1.bam -f BAM -g 800000000 --broad --min-length 1000 --broad-cutoff 0.1 -n H3K4_input_R1 --outdir data/chip-seq/macs_peaks/H3K4_input_R1 > data/chip-seq/macs_peaks/H3K4_input_R1/macs.log 2>&1 || true
epic2 -t data/chip-seq/bam_files/H3K4R1.bam -c data/chip-seq/bam_files/inputR1.bam --chromsizes data/chr_sizes/Lsyl_chr_sizes.tsv -m 0 --output data/chip-seq/epic_peaks/H3K4_input_R1.tsv >data/chip-seq/epic_peaks/H3K4_input_R1.log 2>&1 || true
awk '{print $1"	"$2"	"$3}' data/chip-seq/epic_peaks/H3K4_input_R1.tsv > data/chip-seq/epic_peaks/H3K4_input_R1.bed
macs3 callpeak -t data/chip-seq/bam_files/H3K4R2.bam -c data/chip-seq/bam_files/inputR2.bam -f BAM -g 800000000 --broad --min-length 1000 --broad-cutoff 0.1 -n H3K4_input_R2 --outdir data/chip-seq/macs_peaks/H3K4_input_R2 > data/chip-seq/macs_peaks/H3K4_input_R2/macs.log 2>&1 || true
epic2 -t data/chip-seq/bam_files/H3K4R2.bam -c data/chip-seq/bam_files/

# H3K9me2

mouse antibody - control noAbG

In [4]:
enriched = 'H3K9'
input = ['input', 'noabg']
replicate = ['R1', 'R2']
merging_distance = 50_000

for i in input:
    for r in replicate:
        call_peaks(enriched, i, r, merging_distance)
    intersect_replicates(enriched, i)

macs3 callpeak -t data/chip-seq/bam_files/H3K9R1.bam -c data/chip-seq/bam_files/inputR1.bam -f BAM -g 800000000 --broad --min-length 1000 --broad-cutoff 0.1 -n H3K9_input_R1 --outdir data/chip-seq/macs_peaks/H3K9_input_R1 > data/chip-seq/macs_peaks/H3K9_input_R1/macs.log 2>&1 || true
epic2 -t data/chip-seq/bam_files/H3K9R1.bam -c data/chip-seq/bam_files/inputR1.bam --chromsizes data/chr_sizes/Lsyl_chr_sizes.tsv -m 0 --output data/chip-seq/epic_peaks/H3K9_input_R1.tsv >data/chip-seq/epic_peaks/H3K9_input_R1.log 2>&1 || true
awk '{print $1"	"$2"	"$3}' data/chip-seq/epic_peaks/H3K9_input_R1.tsv > data/chip-seq/epic_peaks/H3K9_input_R1.bed
macs3 callpeak -t data/chip-seq/bam_files/H3K9R2.bam -c data/chip-seq/bam_files/inputR2.bam -f BAM -g 800000000 --broad --min-length 1000 --broad-cutoff 0.1 -n H3K9_input_R2 --outdir data/chip-seq/macs_peaks/H3K9_input_R2 > data/chip-seq/macs_peaks/H3K9_input_R2/macs.log 2>&1 || true
epic2 -t data/chip-seq/bam_files/H3K9R2.bam -c data/chip-seq/bam_files/

# Selection between input sources
For each target, there are two possible sources of control data, either input or no antibody control.
To maximize reliability of the results, we choose the source for each target based on how similar the peak calling is between replicates
Chosen metric to determine the similarity between replicates is the proportion of peaks that do not overlap between replicates.

In [5]:
intersect_replicates('CenH3', 'input')
intersect_replicates('CenH3', 'noaba')
intersect_replicates('H3K4', 'input')
intersect_replicates('H3K4', 'noaba')
intersect_replicates('H3K9', 'input')
intersect_replicates('H3K9', 'noabg')

Proportion of deleted peaks in CenH3_input: 0.2456461961503208
Proportion of deleted peaks in CenH3_noaba: 0.44972067039106145
Proportion of deleted peaks in H3K4_input: 0.16934763181411974
Proportion of deleted peaks in H3K4_noaba: 0.1116214974735875
Proportion of deleted peaks in H3K9_input: 0.3414071510957324
Proportion of deleted peaks in H3K9_noabg: 0.8842105263157894


rename chosen intersected peaks to make them quickly accessible:
```bash
cd data/chip-seq/consensus_peaks
ln CenH3_input_intersected_replicates.bed CenH3_peaks.bed
ln H3K4_noaba_intersected_replicates.bed H3K4_peaks.bed
ln H3K9_input_intersected_replicates.bed H3K9_peaks.bed
```



# Merging close peaks to units - silhouette scores, sum of squares, gap size distribution
[merging optimization notebook](https://github.com/437364/Repeat-based-holocentromeres-of-Luzula-sylvatica/blob/main/notebooks/merging_distance_optimization.ipynb)

# Fixing the issue with epic2 peaks going beyond chromosome bounds
epic2 calls peaks in windows

the last window ends beyond chromosome size

this causes a problem with using the output in bedtools

In [10]:
import pybedtools
bed_path = 'data/chip-seq/consensus_peaks/H3K9_peaks.bed'
chr_sizes = 'data/chr_sizes/Lsyl_chr_sizes.tsv' # format: chrom \t size
bed = pybedtools.BedTool(bed_path)
chromosomewide_windows = bed.window_maker(g=chr_sizes, w=100_000_000_000)
bed_cropped = bed.intersect(b=chromosomewide_windows).saveas(bed_path)
